In [1]:
from pathlib import Path
import os

import torch

from cityslam.videointerface import videointerface, downloader
from cityslam.preprocessing import preprocessing, transitions, create_img_list
from cityslam.mapping import single_video_pipeline
from cityslam.utils import visualization
from natsort import natsorted

# setup necessary paths
base_dir = Path('./demo')

videos_path = base_dir / 'datasets' / 'videos'
images_path = base_dir / 'datasets' / 'images'
queries_path = base_dir / 'datasets' / 'queries'
output_path = base_dir / 'outputs' / 'models'
output_transitions = base_dir / 'outputs' / 'transitions'
output_transitions_cropped = base_dir / 'outputs' / 'transitions_cropped'
image_list_path = base_dir / 'outputs' / 'image_list'

overwrite = False
video_ids = ['gTHMvU3XHBk', 'TZIHy1cZJ-U']
fps = 2

/usr/local/anaconda3/envs/cityslam/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download videos
downloader.main(videos_path, video_ids, format="bv", overwrite=overwrite)

In [ ]:

image_folders = preprocessing.main(
    videos_path, images_path, video_ids, overwrite=overwrite, fps=fps)

In [9]:
path_to_weights = Path('./cityslam/preprocessing')

transitions.main(
    videos_path, video_ids, path_to_weights, output_transitions, output_transitions_cropped, 5*60, 10, fps, 0.5, overwrite_cuts=True)


using device cpu
Already found transitions for video Test Walk 1 Zürich [gTHMvU3XHBk]
video fps: 29.793
Already found transitions for video test walk 2 Zürich [TZIHy1cZJ-U]
video fps: 30.0


In [11]:
create_img_list.create_img_list(output_transitions_cropped, images_path, image_list_path)


demo/outputs/transitions_cropped/TZIHy1cZJ-U_transitions_cropped.txt


FileExistsError: [Errno 17] File exists: 'demo/outputs/image_list/TZIHy1cZJ-U/part00_images.txt'

In [2]:
# Find all scenes
scene_ids = [str(p.relative_to(image_list_path)).split("_images")[0] for p in natsorted(list(image_list_path.glob("**/*_images.txt")))]
print(f"Total scenes: {len(scene_ids)}")

# Filter out the ones that are already done
scene_ids = [scene_id for scene_id in scene_ids if next((output_path / scene_id).glob("**/database.db"), None) is None]
print(f"Scenes left: {len(scene_ids)}")

for scene_id in scene_ids:

    image_list_path = Path(image_list_path) / f"{scene_id}_images.txt"

    reconstruction = single_video_pipeline.main(
        images_path, image_list_path, output_path, video_id=scene_id, window_size=6, num_loc=6, pairing='sequential+retrieval', run_reconstruction=False, overwrite=False)


[2022/06/08 16:01:43 hloc.utils.parsers INFO] Imported 82 images from part00_images.txt
[2022/06/08 16:01:43 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}


Total scenes: 4
Scenes left: 4
getting images...
num images : 82


  0%|          | 0/82 [00:00<?, ?it/s][W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
ERROR: Unexpected segmentation fault encountered in worker.
  0%|          | 0/82 [00:00<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 78547) exited unexpectedly